In [20]:
import time
import selenium as sel
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.firefox.options import Options as firefoxOptions
from bs4 import BeautifulSoup
import pandas as pd

In [25]:
# Configuración de Selenium
def setup_driver():
    chrome_options = Options()
    chrome_options.binary_location = r'E:/Users/1167486/Local/Drivers_web/chrome-win64/chrome.exe'
    chrome_options.add_argument("--headless")  
    chrome_options.add_argument("--disable-notifications")
    chrome_options.add_argument("--lang=es")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    
    #firefox_options = firefoxOptions()
    #firefox_options.add_argument("--headless")  
    #firefox_options.add_argument("--disable-notifications")
    #firefox_options.add_argument("--lang=es")
    #firefox_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

    service = Service('E:/Users/1167486/Local/Drivers_web/chromedriver-win64/chromedriver.exe')  
    #service = Service('E:/Users/1167486/Local/Drivers_web/geckodriver.exe')
    #driver = webdriver.Chrome(service=service, options=firefox_options)
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

In [26]:
driver = setup_driver()

In [ ]:
urlTwitter='https://x.com/RicardoBSalinas/status/1900807572610818356'
urlTwitter = 'https://x.com/f_solorzano/status/1896405011846098960'
#driver.get(urlTwitter)

In [40]:
driver.get(urlTwitter)

In [41]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [42]:
soup.find_all('div', {'data-testid' : 'tweetText'})

[<div class="css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-37j5jr r-1inkyih r-16dba41 r-bnwqim r-135wba7" data-testid="tweetText" dir="auto" id="id__qxyoihzzb9" lang="es" style="color: rgb(15, 20, 25);"><span class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-poiln3">El mejor premio de la noche: Mikey Madison.</span></div>]

In [31]:
def scrape_tweet_comments(tweet_url, max_comments=100):
    driver = setup_driver()
    comments = []
    
    try:
        driver.get(tweet_url)
        time.sleep(5) 
        
        last_height = driver.execute_script("return document.body.scrollHeight")
        comments_collected = 0
        
        while comments_collected < max_comments:
            # Obtener el HTML de la página
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            #comment_elements = soup.find_all('div', {'data-testid': 'tweetText'})
            comment_elements = soup.find_all('div', {'data-testid': 'cellInnerDiv'})
            
            for comment in comment_elements:
                if comment.text not in comments and len(comment.text) > 5:
                    comments.append(comment.text)
                    comments_collected += 1
                    if comments_collected >= max_comments:
                        break
            
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2) 
            
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
            
    except Exception as e:
        print(f"Error durante el scraping: {e}")
    finally:
        driver.quit()
    
    return comments


In [36]:
def scrape_tweet_comments(tweet_url, max_comments=100):
    driver = setup_driver()
    comments = []

    try:
        driver.get(tweet_url)
        time.sleep(5)

        last_height = driver.execute_script("return document.body.scrollHeight")
        comments_collected = 0

        while comments_collected < max_comments:
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # Intenta identificar los comentarios con selectores más específicos
            comment_elements = soup.find_all('div', {'data-testid': 'tweetText'})

            for comment in comment_elements:
                comment_text = comment.text.strip()
                if comment_text and comment_text not in comments:
                    comments.append(comment_text)
                    comments_collected += 1
                    if comments_collected >= max_comments:
                        break

            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)

            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    except Exception as e:
        print(f"Error durante el scraping: {e}")
    finally:
        driver.quit()

    return comments

In [37]:
x  = scrape_tweet_comments(tweet_url=urlTwitter,  max_comments=50)

In [38]:
x

['El mejor premio de la noche: Mikey Madison.']

In [1]:
import requests

In [2]:
params = {
    "screen_name": "f_solorzano",
    "count": 1
}

In [3]:
url_api = "https://api.twitter.com/1.1/statuses/user_timeline.json"

In [4]:
response = requests.get(url_api, params=params)

In [6]:
response.json()

{'errors': [{'code': 215, 'message': 'Bad Authentication data.'}]}